# Wine Classifier

In [ ]:
import numpy as np

import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
np.random.seed(42)

In [ ]:
def train_val_test_split(X, y):
    length_X, length_y = len(X), len(y)
    n = length_X if length_X == length_y else 0

    threshold_1 = round(0.7 * n)
    threshold_2 = round(0.85 * n)

    X_train, y_train = X[:threshold_1], y[:threshold_1]
    X_valid, y_valid = X[threshold_1:threshold_2], y[threshold_1:threshold_2]
    X_test, y_test = X[threshold_2:], y[threshold_2:]

    return (X_train, y_train, X_valid, y_valid, X_test, y_test)


def evaluate_model(y_true, y_pred):
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, average="weighted"),
        "recall": recall_score(y_true, y_pred, average="weighted"),
        "f1": f1_score(y_true, y_pred, average="weighted"),
    }

    print(f"Accuracy: {metrics['accuracy']:.2%}")
    print(f"Precision: {metrics['precision']:.2%}")
    print(f"Recall: {metrics['recall']:.2%}")
    print(f"F1 Score: {metrics['f1']:.2%}")

In [ ]:
data = load_wine()

df = pd.DataFrame(data["data"], columns=data["feature_names"])
df["target"] = data["target"]

df = df.sample(frac=1)

In [ ]:
X = df.drop("target", axis=1)
y = df["target"]

In [ ]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_val_test_split(X, y)

In [ ]:
model_1 = RandomForestClassifier(n_estimators=5)
model_1.fit(X_train, y_train)
evaluate_model(y_test, model_1.predict(X_test))

In [ ]:
model_2 = RandomForestClassifier(n_estimators=100)
model_2.fit(X_train, y_train)
evaluate_model(y_test, model_2.predict(X_test))